# Д/з #4

Используя XGBOOST предсказать размер чаевых в датасете TIPS

In [6]:
import pandas as pd
import sklearn
import xgboost as xgb

def xgb_tips(x_param, y_param):
    x = df_fact[x_param] 
    y = df_fact[y_param]
    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
        x, y, test_size = 0.2)
    XGB = xgb.XGBClassifier(eval_metric = 'rmsle') # rmsle - среднеквадратическая логарифмическая ошибка
    XGB.fit(x_train, y_train) # обучили
    y_pred = XGB.predict(x_test) # предсказали

    df1 = pd.DataFrame({"Тестовые значения tip": y_test, "Предсказанные значения tip": y_pred})
    df1['Триггер'] = df1["Тестовые значения tip"] == df1["Предсказанные значения tip"]
    acuracy_t = df1["Триггер"].sum() / df1["Тестовые значения tip"].count() * 100
    print(f'Предсказанных правильно ТЕСТОВЫХ значений tip: {df1["Триггер"].sum()} \
(из {df1["Тестовые значения tip"].count()}) [точность = {round(acuracy_t, 2)} %]')

    df['Предсказанные tip'] = XGB.predict(x) # еще раз предсказали, но уже по всем значениям
    df['Совпадение'] = df['tip'] == df['Предсказанные tip']
    acuracy_m = df["Совпадение"].sum() / df["tip"].count() * 100
    print(f'Предсказанных правильно ВСЕХ значений tip: {df["Совпадение"].sum()} (из {df["tip"].count()})  \
[точность = {round(acuracy_m, 2)} %]')

    
FILEPATH = ("https://raw.githubusercontent.com/HerrEnot/MISIS/master/notebooks/season_2/tips.csv")
df = pd.read_csv(FILEPATH, parse_dates=True, dayfirst=True)
df_fact = df.apply(lambda x: pd.factorize(x)[0] 
                   if (x.name in ['sex', 'smoker', 'day', 'time']) else x) # закодировали всё
x_param, y_param = ['total_bill', 'sex', 'smoker', 'day', 'time', 'size'], 'tip'
xgb_tips(x_param, y_param)

Предсказанных правильно ТЕСТОВЫХ значений tip: 7 (из 49) [точность = 14.29 %]
Предсказанных правильно ВСЕХ значений tip: 159 (из 244)  [точность = 65.16 %]


In [64]:
df

,total_bill,tip,sex,smoker,day,time,size,Предсказанные tip,Совпадение
0,16.99,1.01,Female,No,Sun,Dinner,2,3.50,False
1,10.34,1.66,Male,No,Sun,Dinner,3,2.00,False
2,21.01,3.50,Male,No,Sun,Dinner,3,3.50,True
3,23.68,3.31,Male,No,Sun,Dinner,2,2.55,False
4,24.59,3.61,Female,No,Sun,Dinner,4,3.61,True
...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,2.05,False
240,27.18,2.00,Female,Yes,Sat,Dinner,2,2.00,True
241,22.67,2.00,Male,Yes,Sat,Dinner,2,2.00,True
242,17.82,1.75,Male,No,Sat,Dinner,2,1.75,True


---------

Хотя точных совпадений всего лишь около *60%*, предсказанные значения варьируются вокруг истины, отклоняясь не особо сильно.
Но всё равно попробуем увеличить точность путем комбинации факторов (фич).

In [25]:
x_param = ['total_bill', 'sex', 'smoker', 'day', 'time']
xgb_tips(x_param, y_param)

Предсказанных правильно ТЕСТОВЫХ значений tip: 9 (из 49) [точность = 18.37 %]
Предсказанных правильно ВСЕХ значений tip: 152 (из 244)  [точность = 62.3 %]


In [31]:
x_param = ['total_bill']
xgb_tips(x_param, y_param)

Предсказанных правильно ТЕСТОВЫХ значений tip: 3 (из 49) [точность = 6.12 %]
Предсказанных правильно ВСЕХ значений tip: 135 (из 244)  [точность = 55.33 %]


In [37]:
x_param = ['total_bill', 'sex', 'day']
xgb_tips(x_param, y_param)

Предсказанных правильно ТЕСТОВЫХ значений tip: 8 (из 49) [точность = 16.33 %]
Предсказанных правильно ВСЕХ значений tip: 149 (из 244)  [точность = 61.07 %]


In [39]:
x_param = ['sex']
xgb_tips(x_param, y_param)

Предсказанных правильно ТЕСТОВЫХ значений tip: 4 (из 49) [точность = 8.16 %]
Предсказанных правильно ВСЕХ значений tip: 33 (из 244)  [точность = 13.52 %]


In [42]:
x_param = ['time', 'smoker', 'day']
xgb_tips(x_param, y_param)

Предсказанных правильно ТЕСТОВЫХ значений tip: 6 (из 49) [точность = 12.24 %]
Предсказанных правильно ВСЕХ значений tip: 39 (из 244)  [точность = 15.98 %]


In [51]:
x_param = ['day', 'size']
xgb_tips(x_param, y_param)

Предсказанных правильно ТЕСТОВЫХ значений tip: 9 (из 49) [точность = 18.37 %]
Предсказанных правильно ВСЕХ значений tip: 46 (из 244)  [точность = 18.85 %]


-------

### Вывод
Как можем заметить, при выборе всех фич для обучения и предсказания, можем получить наиболее точные результаты. 
НО самым значимым параметром является общий счет за посещение (**total_bill**), так как даже с ним одним *XGBoost* может практически с такой же высокой вероятностью предсказать чаевые.